In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# Hyper-parameters
batch_size = 100

In [4]:
# Image preprocessing modules
transform = transforms.ToTensor()

In [5]:
# CIFAR-10 dataset
test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [6]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

In [7]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [8]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [9]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
model = torch.load('resnet.pt')
# model.eval()

In [10]:
print(model)

OrderedDict([('conv.weight', tensor([[[[ 0.1272,  0.0523, -0.1941],
          [-0.0514,  0.0515, -0.1093],
          [-0.1045, -0.0198, -0.0172]],

         [[-0.0468, -0.0433, -0.1852],
          [ 0.0936, -0.0883,  0.1621],
          [-0.0681, -0.1114, -0.1612]],

         [[ 0.0862,  0.1761, -0.2222],
          [ 0.0468,  0.1628,  0.0246],
          [ 0.0448,  0.1119, -0.1234]]],


        [[[-0.0373,  0.0204, -0.0126],
          [-0.0637, -0.0553, -0.0841],
          [ 0.1729,  0.1298,  0.1254]],

         [[-0.1101,  0.1044, -0.0881],
          [ 0.0825, -0.0814, -0.1707],
          [ 0.2046,  0.0751, -0.1708]],

         [[ 0.0637,  0.0183,  0.0773],
          [-0.0280,  0.0818, -0.1099],
          [ 0.1221,  0.0430, -0.1364]]],


        [[[ 0.1532,  0.0007,  0.0156],
          [-0.1140, -0.0937,  0.1531],
          [-0.1737,  0.1345, -0.0146]],

         [[ 0.0298,  0.1600, -0.0307],
          [-0.0398, -0.1767, -0.1683],
          [ 0.0305, -0.1000,  0.1704]],

         [[ 0.0

In [12]:
model.items()

odict_items([('conv.weight', tensor([[[[ 0.1272,  0.0523, -0.1941],
          [-0.0514,  0.0515, -0.1093],
          [-0.1045, -0.0198, -0.0172]],

         [[-0.0468, -0.0433, -0.1852],
          [ 0.0936, -0.0883,  0.1621],
          [-0.0681, -0.1114, -0.1612]],

         [[ 0.0862,  0.1761, -0.2222],
          [ 0.0468,  0.1628,  0.0246],
          [ 0.0448,  0.1119, -0.1234]]],


        [[[-0.0373,  0.0204, -0.0126],
          [-0.0637, -0.0553, -0.0841],
          [ 0.1729,  0.1298,  0.1254]],

         [[-0.1101,  0.1044, -0.0881],
          [ 0.0825, -0.0814, -0.1707],
          [ 0.2046,  0.0751, -0.1708]],

         [[ 0.0637,  0.0183,  0.0773],
          [-0.0280,  0.0818, -0.1099],
          [ 0.1221,  0.0430, -0.1364]]],


        [[[ 0.1532,  0.0007,  0.0156],
          [-0.1140, -0.0937,  0.1531],
          [-0.1737,  0.1345, -0.0146]],

         [[ 0.0298,  0.1600, -0.0307],
          [-0.0398, -0.1767, -0.1683],
          [ 0.0305, -0.1000,  0.1704]],

         [[ 0.0